# 3.6 DG/HDG splitting

When solving unsteady problems with an operator-splitting method it might be benefitial to consider different space discretizations for different operators.

For a problem of the form

$$
  \partial_t u + A u + C u = 0
$$

We consider the operator splitting:

* 1.Step: Given $u^0$, solve $t^n \to t^{n+1}$: $\quad \partial_t u + C u = 0 \Rightarrow u^{\frac12}$
* 2.Step: Given $u^{\frac12}$, solve $t^n \to t^{n+1}$: $\quad \partial_t u + A u = 0 \Rightarrow u^{1}$

This splitting is only first order accurate but allows to take different time discretizations for the substeps 1 and 2. 

In this example we consider the Navier-Stokes problem again (cf. [3.2](../unit-3.2-navierstokes/navierstokes.ipynb)) and want to combine 

* an $H(div)$ -conforming Hybrid DG method (which is a very good discretization for Stokes-type problems) and
* a standard **upwind DG** method for the discretization of the convection

The weak form is: Find $(\mathbf{u},p):[0,T] \to (H_{0,D}^1)^d \times L^2$, s.t.
\begin{align}
\int_{\Omega} \partial_t \mathbf{u} \cdot v + \int_{\Omega} \nu \nabla \mathbf{u} \nabla \mathbf{v} + \mathbf{u} \cdot \nabla \mathbf{u} \mathbf{v} - \int_{\Omega} \operatorname{div}(\mathbf{v}) p &= \int f \mathbf{v}  && \forall \mathbf{v} \in (H_{0,D}^1)^d, \\ 
- \int_{\Omega} \operatorname{div}(\mathbf{u}) q &= 0 && \forall q \in L^2, \\
\quad \mathbf{u}(t=0) & = \mathbf{u}_0
\end{align}


Again, we consider the benchmark setup from http://www.featflow.de/en/benchmarks/cfdbenchmarking/flow/dfg_benchmark2_re100.html . The geometry:

![](http://www.featflow.de/media/dfg_bench2_2d/geometry.png)

The viscosity is set to $\nu = 10^{-3}$.

In [1]:
from netgen import gui
from math import pi
from ngsolve import *
from netgen.geom2d import SplineGeometry
geo = SplineGeometry()
geo.AddRectangle( (0, 0), (2, 0.41), bcs = ("wall", "outlet", "wall", "inlet") )
geo.AddCircle ( (0.2, 0.2), r=0.05, leftdomain = 0, rightdomain = 1, bc = "cyl" )
mesh = Mesh( geo.GenerateMesh(maxh = 0.08) )
order = 3
mesh.Curve(order)

For the HDG formulation we use the product space with

* $BDM_k$: $H(div)$ conforming FE space (degree k)
* Vector facet space: facet functions of degree k (vector valued and only in tangential direction)
* piecewise polynomials up to degree $k-1$ for the pressure

![](resources/stokeshdghdiv.png)

## HDG spaces

In [2]:
V1 = HDiv ( mesh, order = order, dirichlet = "wall|cyl|inlet" )
V2 = FESpace ( "vectorfacet", mesh, order = order, dirichlet = "wall|cyl|inlet" )
Q = L2( mesh, order = order-1)
V = FESpace ([V1,V2,Q])

u, uhat, p = V.TrialFunction()
v, vhat, q = V.TestFunction()

## Stokes discretization
The bilinear form to the HDG discretized Stokes problem is:
\begin{align*}
  K_h((\mathbf{u}_T,\mathbf{u}_F,p),(\mathbf{v}_T,\mathbf{v}_F,q) :=
    & \displaystyle \sum_{T\in\mathcal{T}} \left\{ \int_{T} \nu {\nabla} {\mathbf{u}_T} \! : \! {\nabla} {\mathbf{v}_T} \ d {x} \right. \\ 
    & \qquad \left. - \int_{\partial T} \nu \frac{\partial {\mathbf{u}_T}}{\partial {n} }  [ \mathbf{v} ]_t \ d {s}
    - \int_{\partial T} \nu \frac{\partial {\mathbf{v}_T}}{\partial {n} }  [ \mathbf{u} ]_t \ d {s}
      + \int_{\partial T} \nu \frac{\lambda k^2}{h} [\mathbf{u}]_t [\mathbf{v}]_t \ d {s}  \right\}\\
          & - \int_{\Omega} \operatorname{div}(\mathbf{u}) q \ d {x}
           - \int_{\Omega} \operatorname{div}(\mathbf{v}) p \ d {x}
\end{align*}
where $[\cdot]_t$ is the tangential projection of the jump $(\cdot)_T - (\cdot)_F$.



In [3]:
nu = 0.001 # viscosity
alpha = 4  # SIP parameter

n = specialcf.normal(mesh.dim)
h = specialcf.mesh_size

def tang(vec):
    return vec - (vec*n)*n

deb = dx(element_boundary=True)
a = BilinearForm ( V, symmetric=True)
a += nu * InnerProduct ( Grad(u), Grad(v)) * dx
a += nu * InnerProduct ( Grad(u)*n,  tang(vhat-v) ) * deb
a += nu * InnerProduct ( Grad(v)*n,  tang(uhat-u) ) * deb
a += nu * alpha*order*order/h * InnerProduct ( tang(vhat-v),  tang(uhat-u) ) * deb
a += (-div(u)*q -div(v)*p) * dx
a.Assemble()


The mass matrix is simply
\begin{align*}
  M_h((\mathbf{u}_T,\mathbf{u}_F,p),(\mathbf{v}_T,\mathbf{v}_F,q) := \int_{\Omega} \mathbf{u}_T \cdot \mathbf{v}_T d {x}
\end{align*}

In [4]:
m = BilinearForm(V , symmetric=True)
m += SymbolicBFI( u * v )
m.Assemble()

## Initial conditions
As initial condition we solve the Stokes problem:

In [5]:
gfu = GridFunction(V)

U0 = 1.5
uin = CoefficientFunction( (U0*4*y*(0.41-y)/(0.41*0.41),0) )
gfu.components[0].Set(uin, definedon=mesh.Boundaries("inlet"))

invstokes = a.mat.Inverse(V.FreeDofs(), inverse="umfpack")
gfu.vec.data -= invstokes @ a.mat * gfu.vec

Draw( gfu.components[0], mesh, "velocity" )
Draw( Norm(gfu.components[0]), mesh, "absvel(hdiv)")

## Application of convection

In the operator splitted approach we want to apply only operator applications for the convection part.
Further, we want to do this in a usual DG setting. 
As a model problem we use the following procedure:

* Given $(\mathbf{u},p)$ in HDG space: project into $\hat{\mathbf{u}}$ in usual DG space
* Solve $\partial_t \hat{\mathbf{u}} = C \hat{\mathbf{u}}$ by explicit scheme (involves convection evaluations and mass matrix operations only)
* Solve Unsteady Stokes step with r.h.s. from convection sub-problem. To this end evaluate mixed mass matrix $\int \hat{\mathbf{u}} \cdot \mathbf{v}$ to obtain a functional on the HDG space

For the projection steps we use mixed mass matrices:

* $M_m$ : $HDG \times DG \to \mathbb{R}$
* $M_m^T$ : $DG \times HDG \to \mathbb{R}$
* $M_{DG}$ : $DG \times DG \to \mathbb{R}$ (block diagonal)

To set up mixed mass matrices we use a bilinear form with two different FESpaces. 

We do not assemble the matrices as we will only need the matrix-vector applications of $M_m$, $M_m^T$ and $M_{DG}^{-1}$.

There is a more elegant version (`ConvertL2Operator`) that we discuss below.

### mixed mass matrices

In [6]:
VL2 = VectorL2(mesh, order=order, piola=True)
uL2, vL2 = VL2.TnT()
bfmixed = BilinearForm ( V, VL2, nonassemble=True )
bfmixed += vL2*u*dx
bfmixedT = BilinearForm ( VL2, V, nonassemble=True)
bfmixedT += uL2*v*dx

### convection operator
* convection operation with standard Upwinding
* No set up of the matrix (only interested in operator applications)
* for the advection velocity we use the $H(div)$-conforming velocity (which is pointwise divergence free).

In [7]:
vel = gfu.components[0]
convL2 = BilinearForm(VL2, nonassemble=True )
convL2 += (-InnerProduct(Grad(vL2) * vel, uL2)) * dx
un = InnerProduct(vel,n)
upwindL2 = IfPos(un, un*uL2, un*uL2.Other(bnd=uin))
convL2 += InnerProduct (upwindL2, vL2-vL2.Other()) * dx(skeleton=True)
convL2 += InnerProduct (upwindL2, vL2) * ds(skeleton=True)

### solution of convection steps
We now define the solution of the convection problem for an initial data $u$ in the HDG space. The return value ("res") is $M_m \hat{u}$ where $\hat{u}$ is the solution of several explicit Euler steps of the convection problem
$$
  \partial_t \hat{u} = - M_{DG}^{-1} C \hat{u}
$$

In [8]:
gfuL2 = GridFunction(VL2)
def SolveConvectionSteps(gfuvec, res, tau, steps):
    bfmixed.Apply (gfuvec, gfuL2.vec) 
    VL2.SolveM(gfuL2.vec, CoefficientFunction(1))
    conv_applied = gfuL2.vec.CreateVector()
    for i in range(steps):
        convL2.Apply(gfuL2.vec,conv_applied)
        VL2.SolveM(conv_applied, CoefficientFunction(1))
        gfuL2.vec.data -= tau/steps * conv_applied
    bfmixedT.Apply (gfuL2.vec, res)

## Operator splitting method

In [9]:
# initial values again:
gfu.vec[:] = 0
gfu.components[0].Set(uin, definedon=mesh.Boundaries("inlet"))
gfu.vec.data -= invstokes @ a.mat * gfu.vec
t = 0
tend = 0

In [10]:
tend += 1
substeps = 10
tau = 0.01

mstar = m.mat.CreateMatrix()
mstar.AsVector().data = m.mat.AsVector() + tau * a.mat.AsVector()
inv = mstar.Inverse(V.FreeDofs(), inverse="umfpack")

res = gfu.vec.CreateVector()
while t < tend:
    SolveConvectionSteps(gfu.vec, res, tau, substeps)
    res.data -= mstar * gfu.vec
    gfu.vec.data += inv * res
    t += tau
    print ("\r  t =", t, end="")
    Redraw(blocking=True)

  t = 1.000000000000000775

## `ConvertL2Operator`
Instead of doing the conversion between the spaces by hand, we can use the convenience operator `ConvertL2Operator` that realizes
$ M_{DG}^{-1} \cdot M_m $:

In [11]:
hdiv_to_l2 = V1.ConvertL2Operator(VL2) @ Embedding(V.ndof, V.Range(0)).T
convection = hdiv_to_l2.T @ convL2.mat @ hdiv_to_l2

In [12]:
tend += 0.1
tau = 0.001
from time import sleep
mstar = m.mat.CreateMatrix()
mstar.AsVector().data = m.mat.AsVector() + tau * a.mat.AsVector()
inv = mstar.Inverse(V.FreeDofs(), inverse="umfpack")

res = gfu.vec.CreateVector()
while t < tend:
    res[:]=0
    #SolveConvectionSteps(gfu.vec, res, tau, substeps)
    res.data = m.mat * gfu.vec
    res.data -= tau * convection * gfu.vec
    res.data -= mstar * gfu.vec
    gfu.vec.data += inv * res
    t += tau
    print ("\r  t =", t, end="")
    Redraw(blocking=True)


  t = 1.1009999999999895